In [1]:
import os
import sys

root_dir = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
model_dir = os.path.join(root_dir, "model")
sys.path.append(root_dir)
sys.path.append(model_dir)

from collections import defaultdict
import itertools
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hoho_utils

os.environ['CUDA_LAUNCH_BLOCKING']='1' 

In [2]:
def suffix_time():
    current_time = time.time()
    local_time = time.localtime(current_time)
    time_str = time.strftime('%Y%m%d%H%M%S', local_time)
    return time_str

In [3]:
#test
hoho_utils.INDEXS_2_PIECES

'KARBNPCkarbnpc'

In [4]:
suffix = suffix_time()
suffix

'20230909231508'

In [5]:
data_dir = "../output/data"
file_list = os.listdir(data_dir)
file_count = len(file_list)
item_list = list()

print(f"file_count: {file_count}")

for i, filename in enumerate(file_list):
    file_path = os.path.join(data_dir, filename)
    with open(file_path, "r") as file:
        json_str = file.read()
        matches = json.loads(json_str)
        match_count = len(matches)

        for j, match in enumerate(matches):
#             print(f"match: {match}")
#             break

            base_id =  int(time.time() * 1000) + j
            for k, round in enumerate(match):
                match_id = f"{i}-{j}"
                round_id = f"{base_id}{k}"
                state = round[0]
                action = round[2]
                next_state = round[3]
                chapture_list = round[6]
                chaptures = ','.join(chapture_list)
                chapture_reward = round[7]

                win = 0
                done = round[5]
                if "K" in chapture_list:
                    win = -1
                    done = True
                elif "k" in chapture_list:
                    win = 1
                    done = True

                round_dict = {"match_id": match_id,
                              "round_id": round_id,
                              "state": state,
                              "action":action,
                              "next_state": next_state,
                              "done": done,
                              "chaptures": chaptures,
                              "chapture_reward": chapture_reward,
                              "win": win}
                item_list.append(round_dict)

                if win == 1 or win == -1:
                    break

file_count: 338


In [6]:
data = pd.DataFrame(item_list)

In [7]:
data.head(20)

,match_id,round_id,state,action,next_state,done,chaptures,chapture_reward,win
0,0-0,16942725089600,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
1,0-0,16942725089601,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
2,0-1,16942725089610,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
3,0-1,16942725089611,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
4,0-2,16942725089620,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
5,0-2,16942725089621,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
6,0-3,16942725089630,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
7,0-3,16942725089631,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
8,0-4,16942725089640,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,a0a1,1NBAKABNR/R8/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5...,False,N,-6.28,0
9,0-4,16942725089641,1cBAKABNR/R8/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/7c1...,b2b4,1cBAKABNR/R8/7C1/P1P1P1P1P/1C7/9/p1p1p1p1p/7c1...,False,,0.00,0


In [8]:
len(data[data["win"] == 0]), len(data[data["win"] == 1]), len(data[data["win"] == -1])

(24972, 1, 10052)

In [9]:
# data2 = data.groupby(["match_id"], as_index = False).agg({"action": lambda x: ".".join(x), "chapture_reward": sum})

### hoho_todo: 如果出现重复动作，如何分配？
data2 = data.groupby(["match_id"], as_index = False).agg({"action": lambda x: ",".join(x.unique()), "chapture_reward": sum})
data2.rename(columns = {"action": "action_sequence", "chapture_reward": "chapture_value"}, inplace = True)

In [10]:
data2

,match_id,action_sequence,chapture_value
0,0-0,"f0e1,h2g2",-11.20
1,0-1,"f0e1,h2g2",-11.20
2,0-10,"e0e1,e1f1,i0i2,h2g2,a0a1,b2b5,i2i0,b5h5",-39.07
3,0-11,"f0e1,h2g2",-11.20
4,0-12,"e0e1,b2b4,h2e2,e2g2,c0a2,b0d1,e1e2,a2c4,e2f2,g...",-101.55
...,...,...,...
10049,99-5,"f0e1,h2g2",-11.20
10050,99-6,"e0e1,h2d2,d2d5,b2g2,g0i2,a3a4,a4a5,a0a5,b0c2,g...",-99.55
10051,99-7,"f0e1,h2g2",-11.20
10052,99-8,"f0e1,h2g2",-11.20


In [11]:
data3 = data2.groupby("action_sequence", as_index = False).mean()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_110860\2641885007.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data3 = data2.groupby("action_sequence", as_index = False).mean()


In [12]:
data3

,action_sequence,chapture_value
0,"a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2",-96.17
1,"a0a1,b2b4,a1b1,h2d2,g0e2,b1f1,d0e1,e1f2",-37.61
2,"a0a1,b2b4,a1b1,i3i4,b1e1,e1d1,h2b2,g3g4",-37.61
3,"a0a1,b2b4,a1d1,d1a1,a1f1,h2h5,h0i2,f0e1,f1f6,f...",-105.37
4,"a0a1,b2b4,a1d1,d1a1,f0e1,h2h4,h0i2,c0a2",-62.02
...,...,...
805,"f0e1,g0e2,h2h7,b2b3,a0a1,a1a2",-49.45
806,"f0e1,g0e2,h2h7,b2b9,h7e7,a3a4,a0a2,a2a0,g3g4",-87.71
807,"f0e1,h2g2",-11.20
808,"f0e1,h2h5",-11.20


In [13]:
# 即生成列表List的所有组合
# 如List = [1,2,3]，则返回[], [1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in itertools.combinations(List, i+1)]
    return PS

In [14]:
def subsets(s):
    if len(s) == 1:
        return s
    else:
        sub_channels = []
        for i in range(1, len(s) + 1):
            sub_channels.extend(map(list, itertools.combinations(s, i)))
    return list(map(",".join, map(sorted, sub_channels)))

In [15]:
# demo
test_list = ["e0e1", "e1f1", "i0i2", "h2g2", "a0a1", "b2b5", "i2i0", "b5h5"]
# sorted(test_list)
# subsets(test_list)

In [16]:
def value_function(A, C_values):
    subsets_of_A = subsets(A)
    worth_of_A = 0
    for subset in subsets_of_A:
        print(f"subset: {subset}")
        if subset in C_values:
            worth_of_A += C_values[subset]
    return worth_of_A

In [17]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

In [23]:
#demo
data_demo = data3.head(2)

c_values = data_demo.set_index("action_sequence").to_dict()["chapture_value"]    
channels = []
for i in range(len(data_demo)):
    row = data_demo.iloc[i]
    actions = row["action_sequence"].split(",")
    channels.extend(actions)
    
channels = list(set(channels))
print(f"channels count: {len(channels)}")

power_channels = power_set(channels)
print(c_values)
    
# v_values = {}
# for A in power_channels:
#     v_values[','.join(sorted(A))] = value_function(A, c_values)
# print(f"v_values: {v_values}")


channels count: 16
{'a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2': -96.17, 'a0a1,b2b4,a1b1,h2d2,g0e2,b1f1,d0e1,e1f2': -37.61}


In [21]:
v_values["a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2"]

KeyError: 'a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2'

In [28]:
def calculate_shapley(df, channel_name, conv_name):
    c_values = df.set_index(channel_name).to_dict()[conv_name]    
    df['channels'] = df[channel_name].apply(lambda x: x if len(x.split(",")) == 1 else np.nan)  # hoho: 这里可能会遗漏一些channel
    channels = list(df['channels'].dropna().unique())
    
#     print(f"power_set: {power_set(channels)}")
    
    v_values = {}
    for A in power_set(channels):
        v_values[','.join(sorted(A))] = v_function(A, c_values)
    n = len(channels)
    shapley_values = defaultdict(int)
    
    for channel in channels:
        for A in v_values.keys():
            if channel not in A.split(","):
                cardinal_A = len(A.split(","))  # 没有channel的长度
                A_with_channel = A.split(",")
                A_with_channel.append(channel)
                A_with_channel = ",".join(sorted(A_with_channel))
                weight = (factorial(cardinal_A) * factorial(n - cardinal_A - 1) / factorial(n)) # Weight = |S|!(n-|S|-1)!/n!
                contrib = (v_values[A_with_channel] - v_values[A]) # Marginal contribution = v(S U {i})-v(S)
                shapley_values[channel] += weight * contrib
                
        # Add the term corresponding to the empty set
        shapley_values[channel]+= v_values[channel] / n 
        
    return shapley_values